# Contexte : Analyse du dataset Amazon_Delivery_Dataset pour le pipeline de résilience logistique.

| Variable         | Description courte                                        | Type          | Exemples                  |
|------------------|----------------------------------------------------------|---------------|---------------------------|
| Order_ID         | Identifiant unique de la livraison                       | object        | 'ialx566343618'           |
| Agent_Age        | Âge du livreur                                           | int64         | 37, 34                    |
| Agent_Rating     | Note du livreur                                          | float64       | 4.9, 4.5                  |
| Store_Latitude   | Latitude du point de départ                              | float64       | 22.745049, 12.913041      |
| Store_Longitude  | Longitude du point de départ                             | float64       | 75.892471, 77.683237      |
| Drop_Latitude    | Latitude du point d'arrivée                              | float64       | 22.765049, 13.043041      |
| Drop_Longitude   | Longitude du point d'arrivée                             | float64       | 75.912471, 77.813237      |
| Order_Date       | Date de la commande                                      | object (date) | 2022-03-19                |
| Order_Time       | Heure de la commande                                     | object (heure)| 11:30:00                  |
| Pickup_Time      | Heure de prise en charge                                 | object (heure)| 11:45:00                  |
| Weather          | Conditions météo                                         | object        | Sunny, Stormy, Fog, etc.  |
| Traffic          | Conditions de circulation                                | object        | High, Jam, Low, etc.      |
| Vehicle          | Type de véhicule utilisé                                 | object        | motorcycle, scooter       |
| Area             | Type de zone (urbaine, métropolitaine, etc.)             | object        | Urban, Metropolitian      |
| Delivery_Time    | Temps de livraison en minutes                            | int64         | 120, 165                  |
| Category         | Catégorie de la commande                                 | object        | Clothing, Electronics     |

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Chemin du fichier (toujours utiliser la source extraite, jamais modifier ici)
csv_path = '../data/extracted/Amazon_Delivery_Dataset/amazon_delivery.csv'

In [ ]:
# Chargement du DataFrame
df = pd.read_csv(csv_path)

# Aperçu de la structure
print("Shape (lignes, colonnes) :", df.shape)
print("\nColonnes :", df.columns.tolist())
display(df.head(3))

In [ ]:
# Vérification des types de données et des valeurs manquantes

# Afficher les types de chaque colonne
print("Types de données :")
print(df.dtypes)

print("\nNombre de valeurs manquantes par colonne :")
print(df.isnull().sum())

# Aperçu des stats globales (numériques/catégorielles)
print("\nRésumé statistique rapide :")
display(df.describe(include='all').T)

In [ ]:
# Conversion des colonnes de date et d'heure

# Conversion de Order_Date en datetime
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%Y-%m-%d', errors='coerce')

# Conversion de Order_Time et Pickup_Time en format heure (datetime.time)
df['Order_Time'] = pd.to_datetime(df['Order_Time'], format='%H:%M:%S', errors='coerce').dt.time
df['Pickup_Time'] = pd.to_datetime(df['Pickup_Time'], format='%H:%M:%S', errors='coerce').dt.time

# Vérification rapide
print(df[['Order_Date', 'Order_Time', 'Pickup_Time']].head(5))
print("\nTypes après conversion :")
print(df[['Order_Date', 'Order_Time', 'Pickup_Time']].dtypes)


In [ ]:
# 🧹 Traitement des valeurs manquantes

# 1. Agent_Rating (remplir les valeurs manquantes par la moyenne)
mean_rating = df['Agent_Rating'].mean()
df['Agent_Rating'] = df['Agent_Rating'].fillna(mean_rating)

# 2. Weather (remplir les valeurs manquantes par la modalité la plus fréquente)
mode_weather = df['Weather'].mode()[0]
df['Weather'] = df['Weather'].fillna(mode_weather)

# Vérification
print("Valeurs manquantes après traitement :")
print(df[['Agent_Rating', 'Weather']].isnull().sum())

In [ ]:
# Variables numériques principales
num_vars = ['Agent_Age', 'Agent_Rating', 'Delivery_Time']

for var in num_vars:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[var], kde=True)
    plt.title(f'Distribution de {var}')
    plt.xlabel(var)
    plt.ylabel("Effectif")
    plt.show()

# Variables catégorielles principales
cat_vars = ['Weather', 'Traffic', 'Vehicle', 'Area', 'Category']

for var in cat_vars:
    plt.figure(figsize=(10, 4))
    sns.countplot(data=df, x=var, order=df[var].value_counts().index)
    plt.title(f'Distribution de {var}')
    plt.xlabel(var)
    plt.ylabel("Effectif")
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
# Boxplots croisés pour explorer l’impact des facteurs de risque sur les délais de livraison
plt.figure(figsize=(10, 4))
sns.boxplot(data=df, x='Weather', y='Delivery_Time')
plt.title('Delivery_Time selon la Weather')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 4))
sns.boxplot(data=df, x='Traffic', y='Delivery_Time')
plt.title('Delivery_Time selon Traffic')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 4))
sns.boxplot(data=df, x='Area', y='Delivery_Time')
plt.title('Delivery_Time selon Area')
plt.xticks(rotation=45)
plt.show()

In [ ]:
def recap_delays(df, seuils=[60,120,180,240]):
    nb_total = len(df)
    summary = []
    for seuil in seuils:
        nb = (df['Delivery_Time'] > seuil).sum()
        summary.append({'Seuil (min)': seuil, 'Nb livraisons': nb, 'Taux (%)': 100*nb/nb_total})
    return pd.DataFrame(summary)

display(recap_delays(df))

In [ ]:
# 1. Créer une variable "retard" (=1 si délai > 180 min)
seuil_risk = 120  # À adapter 
df['delivery_risk'] = (df['Delivery_Time'] > seuil_risk).astype(int)

# 2. Calculer le taux de "risque" par contexte
def risk_rate_by(col):
    summary = df.groupby(col)['delivery_risk'].mean().sort_values(ascending=False)
    print(f"\nTaux de livraisons à risque par {col} (>%d min):" % seuil_risk)
    print(summary)
    return summary

risk_rate_by('Weather')
risk_rate_by('Traffic')
risk_rate_by('Area')
risk_rate_by('Vehicle')

In [ ]:
# 1. On retire tous les espaces superflus
df['Traffic'] = df['Traffic'].str.strip()

# 2. On remplace 'NaN', 'nan', '' par np.nan
df['Traffic'] = df['Traffic'].replace(['NaN', 'nan', ''], np.nan)

# 3. On drop les vraies valeurs manquantes
df_clean = df.dropna(subset=['Traffic']).copy()

# 4. On vérifie les valeurs uniques restantes
traffic_order = ['Low', 'Medium', 'High', 'Jam']

# On cast la colonne Traffic en Categorical avec l'ordre métier
df_clean['Traffic'] = pd.Categorical(df_clean['Traffic'], categories=traffic_order, ordered=True)

In [ ]:
cross = df_clean.pivot_table(
    index='Weather',
    columns='Traffic',
    values='delivery_risk',
    aggfunc='mean'
)
# On force l'ordre des colonnes pour la heatmap
cross = cross[traffic_order]  

display(cross)

In [ ]:
# Heatmap des risques croisés Weather x Traffic
risk_crosstab = df_clean.pivot_table(index='Weather', columns='Traffic', values='delivery_risk', aggfunc='mean')
plt.figure(figsize=(10,6))
sns.heatmap(risk_crosstab, annot=True, fmt='.2f', cmap='Reds')
plt.title("Taux de livraisons à risque (>120min) selon Weather x Traffic")
plt.show()


In [ ]:
# Flatten la matrice pour lister les contextes triés par risque décroissant
top_risks = cross.stack().sort_values(ascending=False)
print("TOP contextes à risque (Weather/Traffic) :\n", top_risks.head(5))

top_resilience = cross.stack().sort_values()
print("\nTOP contextes les plus résilients :\n", top_resilience.head(5))

In [ ]:
# Score de risque (taux de retard > 120 min) par zone
area_risk_score = df.groupby('Area')['delivery_risk'].mean().sort_values(ascending=False)
print("Score de risque global par zone (Area) :")
print(area_risk_score)

In [ ]:
area_resilience_score = 1 - area_risk_score


print("Score de résilience par zone (Area) :")
print(area_resilience_score)

In [ ]:
# Ajout du score weather_traffic dans df_clean 
def lookup_weather_traffic_risk(row):
    try:
        return cross.loc[row['Weather'], row['Traffic']]
    except KeyError:
        return np.nan

df_clean['weather_traffic_risk_score'] = df_clean.apply(lookup_weather_traffic_risk, axis=1)

# Ajout du score de risque par zone dans df_clean (pour tout exporter ensemble) --
df_clean['area_risk_score'] = df_clean['Area'].map(area_risk_score)
df_clean['area_resilience_score'] = df_clean['Area'].map(area_resilience_score)

# Export du DataFrame enrichi
df_clean.to_csv('../data/cleaned/amazon_delivery_cleaned.csv', index=False)
print("Export terminé dans /data/cleaned/amazon_delivery_cleaned.csv")

In [ ]:
print(df_clean[['Weather', 'Traffic', 'weather_traffic_risk_score']].head(10))

In [ ]:
# Export du DataFrame enrichi --
df_clean.to_csv('../data/cleaned/amazon_delivery_cleaned.csv', index=False)
print("Export terminé dans /data/cleaned/amazon_delivery_cleaned.csv")

## Synthèse : Facteurs de risques et résilience logistique sur le dernier kilomètre

- Les contextes “Cloudy/Jam” et “Fog/Jam” présentent un taux de retard >120 min de près de 50 %.
- Les situations “Sunny/Low” ou “Windy/Low” sont les plus résilientes (<5 % de retard).
- Le trafic et la météo sont les principaux déterminants du risque, bien plus que la catégorie de produit ou l’âge du livreur.

- Les situations combinant météo défavorable et trafic intense (“Cloudy/Jam”, “Fog/Jam”) affichent un taux de livraisons en retard (>120 min) supérieur à 50 %.

- À l’inverse, les contextes “Sunny/Low” ou “Windy/Low” sont les plus résilients, avec moins de 5 % de retards importants.

- Le trafic et la météo sont les deux principaux déterminants du risque sur le dernier kilomètre, bien avant d’autres facteurs comme la catégorie de commande, l’âge du livreur ou le type de véhicule.

- Chaque livraison est désormais enrichie d’un score de risque contextuel (calculé selon la combinaison météo/traffic observée), ce qui permet de visualiser dynamiquement les “points chauds” dans un dashboard,